<a href="https://colab.research.google.com/github/Uday-ashes-uday/RAG-GEMMA-nutrify/blob/main/implementingRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import requests

In [ ]:
#load pdf

pdf_path="/content/drive/MyDrive/pdf_rag/human-nutrition-text.pdf"

In [ ]:
!pip install pymupdf

In [ ]:
import fitz

from tqdm.auto import tqdm

def text_formatter(text:str):
  cleaned_text=text.replace("\n"," ").strip()
  return cleaned_text


def open_and_read_pdf(pdf_path):
  doc=fitz.open(pdf_path)
  pages_and_texts=[]
  for page_num,page in tqdm(enumerate(doc)):
    text=page.get_text()
    text=text_formatter(text)
    pages_and_texts.append({"page_num":page_num-41,
                            "page_char_count":len(text),
                            "page_word_count":len(text.split(" ")),
                            "page_sent_count":len(text.split(". ")),
                            "page_token_count":len(text)/4,
                            "text":text})

  return pages_and_texts


In [ ]:
pages_and_texts=open_and_read_pdf(pdf_path)

In [ ]:
len(pages_and_texts)

In [ ]:
import random

random.sample(pages_and_texts,k=2)

In [ ]:
import pandas as pd

df=pd.DataFrame(pages_and_texts)

In [ ]:
df.head()

In [ ]:
df.describe()

# Further Text Processing (Splitting pages into sentences)

In [ ]:
from spacy.lang.en import English

#create a pipeline

nlp=English()

nlp.add_pipe("sentencizer")


for item in pages_and_texts:
  item["sentences"]=list(nlp(item['text']).sents)

  item['sentences']=[str(sent) for sent in item['sentences']]

  item['sentence_count_spacy']=len(item['sentences'])




In [ ]:
pages_and_texts[1098+41]

In [ ]:
df=pd.DataFrame(pages_and_texts)
df.head()

In [ ]:
df.describe()

In [ ]:
#Define split size to turn groups of sentences into chunks

def split_text(inp_text,chunk_size=10):
  return [inp_text[i:i+chunk_size] for i in range(0,len(inp_text),chunk_size)]

In [ ]:
for item in tqdm(pages_and_texts):
  item['sentences_chunks']=split_text(item['sentences'])
  item["num_chunks"]=len(item["sentences_chunks"])

In [ ]:
random.sample(pages_and_texts,k=2)

In [ ]:
df=pd.DataFrame(pages_and_texts)
df.head()

In [ ]:
df.describe()

In [ ]:
df.describe().round(2)

In [ ]:
h=random.sample(pages_and_texts,k=1)
h[0]['sentences_chunks']

In [ ]:
h[0]

In [ ]:
for s in h[0]['sentences_chunks']:
  print(''.join(s))

In [ ]:
import re

pages_and_chunks=[]

for item in tqdm(pages_and_texts):

  for sentence_chunk in item['sentences_chunks']:
    chunk_dict={}
    chunk_dict["page_num"]=item["page_num"]
    #join sentences together into paragraph like structure
    joined_sentence_chunk="".join(sentence_chunk).replace("  "," ").strip()
    joined_sentence_chunk=re.sub(r'\.([A-Z])',r'. \1',joined_sentence_chunk)
    chunk_dict['sentence_chunk']=joined_sentence_chunk

    #get some stats on our chunks

    chunk_dict["chunk_char_count"]=len(joined_sentence_chunk)
    chunk_dict["chunk_word_count"]=len([word for word in joined_sentence_chunk.split(" ")])
    chunk_dict["chunk_token_count"]=len(joined_sentence_chunk)/4

    pages_and_chunks.append(chunk_dict)






In [ ]:
len(pages_and_chunks)

In [ ]:
df=pd.DataFrame(pages_and_chunks)
df.head()

In [ ]:
df.describe()

In [ ]:
#filter our dataframe for rows more than 30 tokens

pages_and_chunks_over_min_token_len=df[df["chunk_token_count"]>30].to_dict(orient="records")



In [ ]:
random.sample(pages_and_chunks_over_min_token_len,k=1)

In [ ]:
len(pages_and_chunks_over_min_token_len)

# Embedding our text chunks

In [ ]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 6.2 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
#create some sentences

sentes=["The man in black danced his way to hell.", "The blue curtain hides the view from balcony .", "A chair has back pain"]

In [ ]:
#sentences are encoded/embedded by calling model.encode()

embeddings=emb_model.encode(sentes)

In [ ]:
embed_dict=dict(zip(sentes,embeddings))

In [ ]:
embed_dict["The man in black danced his way to hell."].shape

(768,)

In [ ]:
embed_dict["The blue curtain hides the view from balcony ."].shape

(768,)

In [ ]:
%%time

random.sample(pages_and_chunks_over_min_token_len,k=1)

for i in tqdm(pages_and_chunks_over_min_token_len):
  i["embedding"]=emb_model.encode(i["sentence_chunk"])

NameError: name 'pages_and_chunks_over_min_token_len' is not defined

In [ ]:
#make a list of all chunks

text_chunks=[item['sentence_chunk'] for item in pages_and_chunks_over_min_token_len]

In [ ]:
text_chunks[0]

In [ ]:
text_chunk_embeddings=emb_model.encode(text_chunks,batch_size=32,show_progress_bar=True,convert_to_tensors=True)

Batches:   0%|          | 0/53 [00:00<?, ?it/s]

In [ ]:
len(text_chunk_embeddings)

1680

In [ ]:
pd.DataFrame(pages_and_chunks_over_min_token_len).to_csv("pages_and_chunks_embeddings_df.csv",index=False)

In [ ]:
#load_csv

pd.read_csv("pages_and_chunks_embeddings_df.csv")

# RAG

In [ ]:
#instantiate embedding model

!pip install -U sentence-transformers

from sentence_transformers import SentenceTransformer


emb_model=SentenceTransformer(model_name_or_path="all-mpnet-base-v2",device="cpu")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 5.6 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#load csv

import pandas as pd

data=pd.read_csv("/content/drive/MyDrive/RAG_FROM_SCRATCH/text_chunks_and_embedding.csv")

In [ ]:
data.head()

,page_num,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.00,[ 6.74242675e-02 9.02281404e-02 -5.09548886e-...
1,-38,Human Nutrition: 2020 Edition by University of...,210,30,52.50,[ 5.52156419e-02 5.92139773e-02 -1.66167244e-...
2,-37,Contents Preface University of Hawai‘i at Māno...,766,114,191.50,[ 2.79801842e-02 3.39813754e-02 -2.06426680e-...
3,-36,Lifestyles and Nutrition University of Hawai‘i...,941,142,235.25,[ 6.82566911e-02 3.81275006e-02 -8.46854132e-...
4,-35,The Cardiovascular System University of Hawai‘...,998,152,249.50,[ 3.30264494e-02 -8.49763490e-03 9.57159605e-...


In [ ]:
#RAG goal :Retrieve relevant passages based on a query

#you can turn image into embeddings,sound into embeddings,text into embeddings

import random
import numpy as np
import pandas as pd
import torch

device="cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   page_num           1680 non-null   int64  
 1   sentence_chunk     1680 non-null   object 
 2   chunk_char_count   1680 non-null   int64  
 3   chunk_word_count   1680 non-null   int64  
 4   chunk_token_count  1680 non-null   float64
 5   embedding          1680 non-null   object 
dtypes: float64(1), int64(3), object(2)
memory usage: 78.9+ KB


In [ ]:

#load csv

import pandas as pd

data=pd.read_csv("/content/drive/MyDrive/RAG_FROM_SCRATCH/text_chunks_and_embedding.csv")


data["embedding"]=data["embedding"].apply(lambda x : np.fromstring(x.strip("[]"),sep=" "))


embeddings_co=torch.tensor(np.stack(data["embedding"].to_list(),axis=0),dtype=torch.float32).to(device)

In [ ]:
#create a model

from sentence_transformers import util,SentenceTransformer


emb_model=SentenceTransformer(model_name_or_path="all-mpnet-base-v2",device=device)



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# create a small semantic serach pipeline

## query->query(embedding) ->dot(query_embedding,embedding vectors combined) -> find top 5 scores


### Its important to embed query with same model that was used to create embeddings for passage

In [ ]:
#define the query

query="macronutrients functions"

query_embeddings=emb_model.encode(query,convert_to_tensor=True).to(device)

#get similarity scores

from time import perf_counter as timer

start_timer=timer()

dot_scores=util.dot_score(query_embeddings,embeddings_co)

end_timer=timer()

print(f"INFO: time taken to compute with {len(embeddings_co)} is {end_timer-start_timer:.5f}secs")

INFO: time taken to compute with 1680 is 0.04090secs


In [ ]:
dot_scores.shape

torch.Size([1, 1680])

In [ ]:
top_results_k=torch.topk(dot_scores,k=5)
top_results_k

torch.return_types.topk(
values=tensor([[0.6926, 0.6738, 0.6646, 0.6536, 0.6473]], device='cuda:0'),
indices=tensor([[42, 47, 41, 51, 46]], device='cuda:0'))

# Similarity Search

### dot product and cosine similarity

In [ ]:
#functionalize dot scores


def retrieve_relevant_resources(query:str,
                                embedding:torch.Tensor,
                                model:SentenceTransformer,
                                n_resources:int,
                                print_timer:bool=True):

  query_emb=model.encode(query,convert_to_tensor=True).to(device)

  start_timer=timer()

  dot_scores=util.dot_score(query_emb,embedding)

  top_results_k=torch.topk(dot_scores,k=n_resources)

  end_timer=timer()

  print(f"[INFO] this results in time taken {end_timer-start_timer:.5f}")

  scores,indices=top_results_k

  return scores,indices

In [ ]:
import textwrap
def print_top_results_and_scores(query:str,
                                 embeddings:torch.Tensor,
                                 model:SentenceTransformer,
                                 n_resources:int,
                                 pages_and_chunks,print_timer=True):

  scores,indices=retrieve_relevant_resources(query=query,
                                             embedding=embeddings,
                                             model=model,
                                             n_resources=n_resources)

  for i,j in zip(scores[0],indicies[0]):
      print(f'{i.item():.3f}')
      text=data.loc[j.item()]['sentence_chunk']
      print(textwrap.fill(text))
      print('\n')

# Getting an LLM for local generation

In [ ]:
!pip install transformers bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.9 MB/s eta 0:00:00


In [ ]:
!pip install huggingface_hub

# Hugging face setup

In [ ]:
# prompt: import HF token from google colab secret keys
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

import torch
import transformers
from transformers import AutoTokenizer,AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available
from transformers import BitsAndBytesConfig



from huggingface_hub import login

login(token=hf_token, add_to_git_credential=True)

device="cuda" if torch.cuda.is_available() else "cpu"


quantization_config=BitsAndBytesConfig(load_in_4bit=True,
                                       bnb_4bit_compute_dtype=torch.float16)


if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0)(0)>=8):
    attn_implementation="flash_attention_2"
else:
    attn_implementation="sdpa"



model_id="google/gemma-2-2b-it"

#instantiate tokenizer

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")

#instantiate the model

llm_model=AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id,
                                               torch_dtype=torch.float16,
                                               #quantization_config=False,
                                               low_cpu_mem_usage=False,
                                               attn_implementation=attn_implementation)

llm_model.to(device)


Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2SdpaAttention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), 

In [ ]:
llm_model

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2SdpaAttention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), 

In [ ]:
def get_model_num_params(model:torch.nn.Module):
  return sum([param.numel() for param in model.parameters()])

In [ ]:
get_model_num_params(llm_model)

2614341888

# Generate response from LLM

In [ ]:
input_text="What are macronutrients? What role do they play in the human body?"


dialogue_template=[
    {"role":"user","content":input_text}
]

In [ ]:
#apply the chat template

prompt=tokenizer.apply_chat_template(conversation=dialogue_template,
                                     tokenize=False,
                                     add_generation_prompt=True)

In [ ]:
print(f"Prompt is {prompt}")

Prompt is <bos><start_of_turn>user
What are macronutrients? What role do they play in the human body?<end_of_turn>
<start_of_turn>model



In [ ]:
%%time

input_id=tokenizer(prompt,return_tensors="pt").to(device)

CPU times: user 1.98 ms, sys: 0 ns, total: 1.98 ms
Wall time: 7.01 ms


In [ ]:
input_id

{'input_ids': tensor([[     2,      2,    106,   1645,    108,   1841,    708, 186809, 184592,
         235336,   2439,   4731,    749,    984,   1554,    575,    573,   3515,
           2971, 235336,    107,    108,    106,   2516,    108]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]], device='cuda:0')}

In [ ]:
#generate output from local llm

output=llm_model.generate(**input_id,max_new_tokens=256)

print(output[0])

tensor([     2,      2,    106,   1645,    108,   1841,    708, 186809, 184592,
        235336,   2439,   4731,    749,    984,   1554,    575,    573,   3515,
          2971, 235336,    107,    108,    106,   2516,    108,   1620,  97586,
        184592, 235292,    714,  14427, 235303, 235256,  31219,    109,  12298,
          1695, 184592,    708,    573,  37132,    861,   2971,   4026,    575,
          2910,  15992,    577,   1411,  10338, 235265,   2365,   3658,    573,
          4134,    578,   4547,  13854,    604,   5115, 235269,  12158, 235269,
           578,   8691,   2962, 235265, 235248,    109,   3493,    708,   2149,
          1872, 186809, 184592, 235292,    109,    688, 235274, 235265, 110165,
         56227,  66058,    109, 235287,   5231,  11071,  66058,    714,   7920,
          4303,    576,   4134,    604,    573,   2971, 235265,   2365,    708,
         10127,   1706,   1280,  30859, 235269,    948,  47043,    861,   5999,
           578,  29998, 235265,    108, 

In [ ]:
#decode the output tokens

decoded_output=tokenizer.decode(output[0])

In [ ]:
print(decoded_output)

<bos><bos><start_of_turn>user
What are macronutrients? What role do they play in the human body?<end_of_turn>
<start_of_turn>model
## Macronutrients: The Body's Fuel

Macronutrients are the nutrients your body needs in large amounts to function properly. They provide the energy and building blocks for growth, repair, and overall health. 

There are three main macronutrients:

**1. Carbohydrates:**

* **Role:** The primary source of energy for the body. They are broken down into glucose, which fuels your cells and organs.
* **Types:**
    * **Simple carbohydrates:** Found in sugars like glucose, fructose, and lactose. They are quickly absorbed into the bloodstream, providing a quick burst of energy.
    * **Complex carbohydrates:** Found in starches and fiber. They are digested more slowly, providing sustained energy release.
* **Sources:** Fruits, vegetables, grains, legumes, dairy products.

**2. Proteins:**

* **Role:** Essential for building and repairing tissues, making enzymes and

# Augment prompt and context

In [ ]:
def ask_me_any(query:str,
        chunks_and_embs,
        model=llm_model,
        temperature:float=0.7,
        max_new_tokens:int=256,
        format_answer_text=True,
        return_answer_only=True,
        emb_model=emb_model):


  """Takes a query, finds the relevant resources/ context and generates an
   answer to the query based on the relevant resources"""


  scores,indices=retrieve_relevant_resources(query=query,
                                             embedding=embeddings_co,
                                             model=emb_model,
                                             n_resources=5)


  #scores,indices=retrieve_relevant_resources(query=query,
  #                                            embedding=chunks_and_embs,
  #                                            model=model,n_resources=5)

  context="-"+"\n".join([chunks_and_embs.loc[ind.item()]['sentence_chunk'] for ind in indices[0]])

  base_prompt_in=f"""Based on the following context items, Please answer the following query,
                     context items:{context}, query:{query} Answer: """


  dialogue_prompt=[
                    {"role":"user",
                     "content":f"""Based on the following context items, Please answer the following query,
                     context items:{context}, query:{query} Answer: """}
                   ]


  tp=tokenizer.apply_chat_template(conversation=dialogue_prompt,
                              tokenize=False,
                              add_generation_prompt=True)


  input_ids=tokenizer(tp,return_tensors="pt").to(device)

  outputs=llm_model.generate(**input_ids,
                           temperature=0.7,
                           do_sample=True,
                           max_new_tokens=256)

  out_d=tokenizer.decode(outputs[0])

  out_r=out_d.replace(tp,"").replace("bos","").replace("*"," ")

  return out_r


In [ ]:
scores,indices=retrieve_relevant_resources(query=query,
                                             embedding=embeddings_co,
                                             model=emb_model,
                                             n_resources=5)

[INFO] this results in time taken 0.00014


In [ ]:
indices

tensor([[42, 47, 41, 51, 46]], device='cuda:0')

In [ ]:
scores

tensor([[0.6926, 0.6738, 0.6646, 0.6536, 0.6473]], device='cuda:0')

In [ ]:
fg=ask_me_any(query="macronutrients functions",
           chunks_and_embs=data)

#print(fg.replace(tp,"").replace("bos","").replace("*"," "))
#print(fg.replace(tp,"").replace("bos","").replace("*"," "))


#print(textwrap.fill(fg))

[INFO] this results in time taken 0.00014


In [ ]:
print(fg)

<>The provided text details the functions of macronutrients and gives you a clear understanding of their roles in the body. Here's a summary of their key functions:

  Macronutrients:  

    Carbohydrates:    
        Primary function:   Provide energy (through chemical bonds) for cellular processes. 
        Further breakdown:    
          Broken down into glucose, which is the body's primary energy source.
          Energy for basic functions like muscle contraction, brain function, and organ function.

    Lipids:   
        Primary function:   
          Primarily energy storage.
          Insulation and protection of organs.
          Building blocks for cell membranes.
          Regulate hormone production.
          Aids in absorption of fat-soluble vitamins.

    Proteins:   
        Primary function:  
          Structure: Build and repair tissues (muscles, bones, skin, organs).
          Enzymes: Catalyze (speed up) chemical reactions in the body.
          Hormones: Regulat